In [10]:
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
from nltk.corpus import stopwords

In [11]:
# Descargar las stopwords en español
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Configuración de MongoDB
MONGO_URI = "mongodb://localhost:27017/"  # Cambia esto si usas una configuración diferente
DATABASE_NAME = "tesis"
COLLECTION_NAME = "ciberbullying"

In [13]:
# --- Análisis de Procesamiento de Lenguaje Natural (PLN) ---

# Cargar datos desde MongoDB
def fetch_data_from_mongo(db_name, collection_name):
    """
    Extrae datos de una colección MongoDB y los convierte en un DataFrame.
    """
    client = MongoClient(MONGO_URI)
    db = client[db_name]
    collection = db[collection_name]
    data = pd.DataFrame(list(collection.find()))
    client.close()
    return data

data = fetch_data_from_mongo(DATABASE_NAME, COLLECTION_NAME)

In [14]:
# Limpieza de datos
def clean_data(data, text_column):
    """
    Limpia el DataFrame eliminando o reemplazando valores nulos en la columna de texto.

    Parameters:
        data (DataFrame): Datos con texto y etiquetas.
        text_column (str): Nombre de la columna de texto.

    Returns:
        DataFrame: Datos limpios.
    """
    data[text_column] = data[text_column].fillna("")  # Reemplazar NaN con cadena vacía
    return data

# Limpieza de los datos antes del preprocesamiento
data = clean_data(data, "texto")

In [15]:
# Preprocesamiento del texto
def preprocess_text(data, text_column, label_columns):
    """
    Preprocesa los textos y divide en conjuntos de entrenamiento y prueba.

    Parameters:
        data (DataFrame): Datos con columnas de texto y etiquetas.
        text_column (str): Nombre de la columna con texto.
        label_columns (list): Nombres de las columnas de etiquetas.

    Returns:
        X_train, X_test, y_train, y_test: Conjuntos de datos divididos.
    """
    vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
    X = vectorizer.fit_transform(data[text_column])
    y = data[label_columns]
    return train_test_split(X, y, test_size=0.2, random_state=42)

text_column = "texto"  # Columna de texto
label_columns = [
    "insulto", "amenaza", "rumores_difamatorios", "comentarios_humillantes", "lenguaje_discriminatorio",
    "acoso_sexual", "manipulacion_emocional", "lenguaje_vulgar", "acoso_general", "redes_sociales",
    "suicidio_autolesion"
]  # Columnas de etiquetas

In [16]:
# Preprocesamiento
X_train, X_test, y_train, y_test = preprocess_text(data, text_column, label_columns)

In [17]:
# --- Modelo de Machine Learning: Regresión Logística ---
lr_base_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model = MultiOutputClassifier(lr_base_model)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

In [18]:
# Evaluación del modelo por etiqueta
print("\nReporte de métricas para Regresión Logística (multietiqueta):")
for i, column in enumerate(label_columns):
    print(f"\nEtiqueta: {column}")
    print(classification_report(y_test.iloc[:, i], lr_predictions[:, i]))


Reporte de métricas para Regresión Logística (multietiqueta):

Etiqueta: insulto
              precision    recall  f1-score   support

       False       0.97      1.00      0.98      8615
        True       1.00      0.77      0.87      1347

    accuracy                           0.97      9962
   macro avg       0.98      0.88      0.93      9962
weighted avg       0.97      0.97      0.97      9962


Etiqueta: amenaza
              precision    recall  f1-score   support

       False       0.99      1.00      0.99      9840
        True       1.00      0.16      0.28       122

    accuracy                           0.99      9962
   macro avg       0.99      0.58      0.64      9962
weighted avg       0.99      0.99      0.99      9962


Etiqueta: rumores_difamatorios
              precision    recall  f1-score   support

       False       0.97      1.00      0.99      9369
        True       1.00      0.52      0.69       593

    accuracy                           0.97      

In [20]:
# Evaluación general
lr_accuracy = accuracy_score(y_test.values, lr_predictions)
print(f"\nAccuracy para Regresión Logística: {lr_accuracy:.2f}")


Accuracy para Regresión Logística: 0.86
